# Spotify Song Suggestions Using KNN

In [15]:
# Cell 1
import pandas as pd
import numpy as np


In [77]:
# Cell 2
df = pd.read_csv("../datasets/dataset.csv", index_col=0)  # change filename/path if needed
df

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [78]:
dup_count = df["track_id"].duplicated().sum()
print("Duplicate track_id count:", dup_count)

Duplicate track_id count: 24259


In [79]:
dup_rows_count = df["track_name"].duplicated().sum()
print("Duplicate track_name rows (excluding first occurrences):", dup_rows_count)

Duplicate track_name rows (excluding first occurrences): 40391


In [80]:
# df loaded already
cols = df.columns.tolist()
identical_ids = df.groupby("track_id")[cols].apply(lambda g: (g.drop(columns="track_id").nunique().eq(1).all()))
identical_ids = identical_ids[identical_ids].index.to_list()
len(identical_ids), identical_ids[:20]

(73441,
 ['0000vdREvCVMxbQTkS888c',
  '000CC8EParg64OmTxVnZ0p',
  '000Iz0K615UepwSJ5z2RE5',
  '000RDCYioLteXcutOjeweY',
  '000qpdoc97IMTBvF8gwcpy',
  '0017XiMkqbTfF2AUOzlhj6',
  '001pyq8FLNSL1C8orNLI0b',
  '002qpSULhHAw6DGqFxbaO1',
  '002uYDBLOvJz21C4FuArDS',
  '003lo4y8gOylAqDs2scLx2',
  '004G9E3EZhxxn5aE9yEQqx',
  '004iWPkSRbvOEvAPLWHl9M',
  '0051nJ5xbRu8kuqPTYa9l7',
  '005NLlv3rClEVqeoSyYTHU',
  '005U5ZHLoA4OEkWewAa32S',
  '0068lzo1xXa9ED8ThypHU1',
  '006ATYzgynEKIPgVaT5LQM',
  '006Bi4j0yzwOc3y69GOlYV',
  '006c9li2Mybyg5vm6doEfO',
  '0072MKNpMeJNx8aRfTNfQW'])

In [81]:
dup_count = df.duplicated().sum()
print("Exact duplicate rows (excluding first occurrences):", dup_count)

Exact duplicate rows (excluding first occurrences): 450


In [82]:
dup_all = df.duplicated(keep=False).sum()
print("Total rows that are part of any duplicate group:", dup_all)

Total rows that are part of any duplicate group: 894


In [83]:
df[df.duplicated(keep=False)].sort_values(list(df.columns)).head(100)


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
52714,00JZ83w0Qm09f4PwWj06sM,George Jones,With Love,A Good Year For The Roses,12,190546,False,0.491,0.3340,11,-9.684,1,0.0287,0.6590,0.000016,0.1160,0.2490,91.674,4,honky-tonk
52766,00JZ83w0Qm09f4PwWj06sM,George Jones,With Love,A Good Year For The Roses,12,190546,False,0.491,0.3340,11,-9.684,1,0.0287,0.6590,0.000016,0.1160,0.2490,91.674,4,honky-tonk
39275,02KmEChUwcjxG3G29kbLFT,Hans Zimmer;Henning Lohner;Martin Tillman;Fiac...,Hans Zimmer: Epic Scores,Shelter Mountain,16,250520,False,0.144,0.2620,9,-21.228,1,0.0641,0.8760,0.755000,0.1440,0.0345,94.430,4,german
39307,02KmEChUwcjxG3G29kbLFT,Hans Zimmer;Henning Lohner;Martin Tillman;Fiac...,Hans Zimmer: Epic Scores,Shelter Mountain,16,250520,False,0.144,0.2620,9,-21.228,1,0.0641,0.8760,0.755000,0.1440,0.0345,94.430,4,german
93331,02MRylJ1WAgxzdqfNfdIsR,Oleg Pogudin,Любовь и разлука. Песни Исаака Шварца,"Город пышный, город бедный",0,114600,False,0.446,0.1210,4,-12.737,0,0.0414,0.9370,0.000000,0.1250,0.1800,107.573,3,romance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105810,0xchuOuU0MqjKaUrzIPrCv,Pepper and Po,Breaking News,Subway Signs,26,183007,False,0.792,0.0414,8,-14.461,1,0.0663,0.7480,0.929000,0.0902,0.4180,79.985,4,study
105866,0xchuOuU0MqjKaUrzIPrCv,Pepper and Po,Breaking News,Subway Signs,26,183007,False,0.792,0.0414,8,-14.461,1,0.0663,0.7480,0.929000,0.0902,0.4180,79.985,4,study
59703,0xsbEfKgnxpXs1IRAbEnUY,DSM,Nowruz,Nowruz,0,259970,False,0.236,0.0463,0,-26.831,0,0.0420,0.9270,0.815000,0.1030,0.0325,121.532,3,iranian
59806,0xsbEfKgnxpXs1IRAbEnUY,DSM,Nowruz,Nowruz,0,259970,False,0.236,0.0463,0,-26.831,0,0.0420,0.9270,0.815000,0.1030,0.0325,121.532,3,iranian


In [84]:
df_dedup = df.drop_duplicates(keep="first").reset_index(drop=True)
df_dedup

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113545,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113546,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113547,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113548,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [85]:
dup_all = df_dedup.duplicated(keep=False).sum()
print("Total rows that are part of any duplicate group:", dup_all)

Total rows that are part of any duplicate group: 0


In [86]:
dup_count = df_dedup.duplicated(subset=["track_id","artists","album_name","track_name"]).sum()
print("Duplicate rows (excluding first occurrences):", dup_count)

Duplicate rows (excluding first occurrences): 23809


In [87]:
df_unique = df_dedup.drop_duplicates(subset=["track_id","artists","album_name","track_name"], keep="first").reset_index(drop=True)
df_unique

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89736,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
89737,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
89738,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
89739,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [88]:
# count duplicate track_id values in df_dedup
dup_count = df_unique["track_name"].duplicated().sum()
print("Duplicate track_name rows (excluding first occurrences):", dup_count)
dup_count = df_unique["track_id"].duplicated().sum()
print("Duplicate track_id rows (excluding first occurrences):", dup_count)
dup_count = df_unique["album_name"].duplicated().sum()
print("Duplicate album name rows (excluding first occurrences):", dup_count)
dup_count = df_unique[["album_name", "artists", "track_name"]].duplicated().sum()
print("Duplicated across album name, track name and artists rows (excluding first occurrences):", dup_count)



Duplicate track_name rows (excluding first occurrences): 16132
Duplicate track_id rows (excluding first occurrences): 0
Duplicate album name rows (excluding first occurrences): 43151
Duplicated across album name, track name and artists rows (excluding first occurrences): 361


In [89]:
df_unique2 = df_dedup.drop_duplicates(subset=["artists","album_name","track_name"], keep="first").reset_index(drop=True)
df_unique2

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89375,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
89376,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
89377,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
89378,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [90]:
# count duplicate track_id values in df_dedup
dup_count = df_unique2["track_name"].duplicated().sum()
print("Duplicate track_name rows (excluding first occurrences):", dup_count)
dup_count = df_unique2["track_id"].duplicated().sum()
print("Duplicate track_id rows (excluding first occurrences):", dup_count)
dup_count = df_unique2["album_name"].duplicated().sum()
print("Duplicate album name rows (excluding first occurrences):", dup_count)
dup_count = df_unique2[["album_name", "artists", "track_name"]].duplicated().sum()
print("Duplicated across album name, track name and artists rows (excluding first occurrences):", dup_count)
dup_count = df_unique2[["track_name", "duration_ms", "explicit", "danceability", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature" ]].duplicated().sum()
print("Duplicated across track name and song values (excluding first occurrences):", dup_count)
df_unique3 = df_unique2.drop_duplicates(subset=["track_name", "duration_ms", "explicit", "danceability", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature" ], keep="first").reset_index(drop=True)
df_unique3


Duplicate track_name rows (excluding first occurrences): 15771
Duplicate track_id rows (excluding first occurrences): 0
Duplicate album name rows (excluding first occurrences): 42790
Duplicated across album name, track name and artists rows (excluding first occurrences): 0
Duplicated across track name and song values (excluding first occurrences): 5679


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83696,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
83697,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
83698,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
83699,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [91]:
dup_count = df_unique3[["track_name", "duration_ms", "explicit", "danceability", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature" ]].duplicated().sum()
print("Duplicated across track name and song values (excluding first occurrences):", dup_count)
dup_count = df_unique3[["track_name", "explicit", "danceability", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature" ]].duplicated().sum()
print("Duplicated across track name and song values less duration (excluding first occurrences):", dup_count)

Duplicated across track name and song values (excluding first occurrences): 0
Duplicated across track name and song values less duration (excluding first occurrences): 11


In [92]:
df_unique4 = df_unique3.drop_duplicates(subset=["track_name", "explicit", "danceability", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"], keep="first").reset_index(drop=True)
df_unique4


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83685,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
83686,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
83687,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
83688,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [93]:
dup_count = df_unique4[["track_name", "explicit", "danceability", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature" ]].duplicated().sum()
print("Duplicated across track name and song values less duration (excluding first occurrences):", dup_count)
dup_count = df_unique4[["track_name", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "tempo"]].duplicated().sum()
print("Duplicated across track name and song values - less valence, danceability, time signature (excluding first occurrences):", dup_count)

Duplicated across track name and song values less duration (excluding first occurrences): 0
Duplicated across track name and song values - less valence, danceability, time signature (excluding first occurrences): 10


In [94]:
df_unique5 = df_unique4.drop_duplicates(subset=["track_name", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "tempo"], keep="first").reset_index(drop=True)
df_unique5

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83675,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
83676,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
83677,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
83678,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [95]:

dup_count = df_unique5[["track_name", "energy","loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "tempo"]].duplicated().sum()
print("Duplicated across track name and song values - less valence, danceability, time signature (excluding first occurrences):", dup_count)

Duplicated across track name and song values - less valence, danceability, time signature (excluding first occurrences): 0


In [96]:
df_unique5.isna().any()


track_id            False
artists              True
album_name           True
track_name           True
popularity          False
duration_ms         False
explicit            False
danceability        False
energy              False
key                 False
loudness            False
mode                False
speechiness         False
acousticness        False
instrumentalness    False
liveness            False
valence             False
tempo               False
time_signature      False
track_genre         False
dtype: bool

In [97]:
df_unique5[df_unique5.isna().any(axis=1)]


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
51855,1kR4gIb7nGxHPI3D2ifs59,NaN,NaN,NaN,0,0,False,0.501,0.583,7,-9.46,0,0.0605,0.69,0.00396,0.0747,0.734,138.391,4,k-pop


In [98]:
df_clean = df_unique5.dropna()


In [99]:
df_clean.isna().sum()


track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [100]:
df_clean.describe

<bound method NDFrame.describe of                      track_id                 artists  \
0      5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1      4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2      1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3      6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4      5vjLSffimiIP26QG5WcN2K        Chord Overstreet   
...                       ...                     ...   
83675  2C3TZjDRiAzdyViavDJ217           Rainy Lullaby   
83676  1hIz5L4IB9hN3WRYPOCGPw           Rainy Lullaby   
83677  6x8ZfSoqDjuNa5SVP5QjvX           Cesária Evora   
83678  2e6sXL2bYv4bSz6VTdnfLs        Michael W. Smith   
83679  2hETkH7cOfqmz3LqZDHZf5           Cesária Evora   

                                              album_name  \
0                                                 Comedy   
1                                       Ghost (Acoustic)   
2                                         To Begin Again   
3      Crazy Rich Asians (Original Motion

In [101]:
df_clean.info()


<class 'pandas.DataFrame'>
Index: 83679 entries, 0 to 83679
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          83679 non-null  str    
 1   artists           83679 non-null  str    
 2   album_name        83679 non-null  str    
 3   track_name        83679 non-null  str    
 4   popularity        83679 non-null  int64  
 5   duration_ms       83679 non-null  int64  
 6   explicit          83679 non-null  bool   
 7   danceability      83679 non-null  float64
 8   energy            83679 non-null  float64
 9   key               83679 non-null  int64  
 10  loudness          83679 non-null  float64
 11  mode              83679 non-null  int64  
 12  speechiness       83679 non-null  float64
 13  acousticness      83679 non-null  float64
 14  instrumentalness  83679 non-null  float64
 15  liveness          83679 non-null  float64
 16  valence           83679 non-null  float64
 17  tempo    

In [102]:
df_clean.to_csv("df_clean.csv", index=False)


In [103]:
df_clean.to_csv("df_clean.csv", index=False)
from IPython.display import FileLink
FileLink("df_clean.csv")


c:\Users\SamF\CODE INSTITUTE LMS\3. Projects\Project 3 Github\jupyter_notebooks\df_clean.csv

In [16]:
featured_df = pd.read_csv(r'..\datasets\sam_df_clean_featured_new.csv')

In [17]:
# Creating a new boolean column where it indicates if a track is featured or not. True if featured, False if not.
# Tracks with ';' have more than one artist, indicating a featured track.
featured_df['is_featured'] = featured_df['artists'].apply(lambda x: ';' in x)
featured_df.head(10)

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,genre_subcategory,popularity_bin,popularity_top_10,energy_code,loudness_code,dance_level,music_valence,tempo_bin,num_artists,is_featured
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,niche-mood,Chart Climbers,0,Chill,Medium,Bump,happy/content,slow,1,False
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,niche-mood,Radio Hits,0,Chill,Quiet,Groove,sad/angry,slow,1,False
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,niche-mood,Radio Hits,0,Chill,Low,Groove,sad/angry,slow,2,True
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,niche-mood,Chart Climbers,0,Chill,Quiet,Sway,sad/angry,fast,1,False
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,niche-mood,Bangers,0,Chill,Low,Bump,sad/angry,mid,1,False
5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.4810,6,...,niche-mood,Radio Hits,0,Chill,Low,Bump,happy/content,slow,1,False
6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,2,...,niche-mood,Chart Climbers,0,Chill,Low,Groove,sad/angry,mid,2,True
7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,11,...,niche-mood,Chart Climbers,0,Chill,Low,Bump,happy/content,fast,1,False
8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,0,...,niche-mood,Chart Climbers,0,Chill,Low,Bump,happy/content,mid,2,True
9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,56,205594,False,0.442,0.6320,1,...,niche-mood,Radio Hits,0,Chill,Medium,Groove,sad/angry,slow,1,False


In [19]:
# updated sam_df_clean_featured with is_featured column
featured_df.to_csv(r'..\datasets\sam_df_clean_featured_new.csv', index=False)